<a href="https://colab.research.google.com/github/arjunkpreman/Transfer-Learning-Project/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Load pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87910968/87910968 [==============================] - 4s 0us/step


In [6]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [7]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)  # num_classes is the number of classes in your task

In [8]:

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

Found 3015 images belonging to 10 classes.


In [11]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 921 images belonging to 10 classes.


In [12]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=5
)

Epoch 1/5
94/94 [==============================] - 2366s 25s/step - loss: 1.2369 - accuracy: 0.5716 - val_loss: 0.7810 - val_accuracy: 0.7388
Epoch 2/5
94/94 [==============================] - 23s 245ms/step - loss: 0.5658 - accuracy: 0.8103 - val_loss: 0.5643 - val_accuracy: 0.8036
Epoch 3/5
94/94 [==============================] - 24s 250ms/step - loss: 0.4086 - accuracy: 0.8723 - val_loss: 0.8258 - val_accuracy: 0.7132
Epoch 4/5
94/94 [==============================] - 23s 244ms/step - loss: 0.3518 - accuracy: 0.8800 - val_loss: 0.5033 - val_accuracy: 0.8114
Epoch 5/5
94/94 [==============================] - 23s 243ms/step - loss: 0.2517 - accuracy: 0.9249 - val_loss: 0.4839 - val_accuracy: 0.8382


In [13]:
# Save the trained model
model.save('/content/drive/MyDrive/Dataset/InceptionV3.h5')

In [14]:
model.evaluate(val_generator)

29/29 [==============================] - 8s 266ms/step - loss: 0.4791 - accuracy: 0.8393


[0.4790717363357544, 0.8393051028251648]

In [15]:
# Recreate the exact same model, including its weights and the optimizer
InceptionV3model = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/InceptionV3.h5')

# Show the model architecture
# vgg_model.summary()

In [16]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

Found 3015 images belonging to 10 classes.


In [17]:
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 921 images belonging to 10 classes.


In [18]:
# Train the model
InceptionV3model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32,
    epochs=5
)

Epoch 1/5
94/94 [==============================] - 28s 260ms/step - loss: 0.2150 - accuracy: 0.9376 - val_loss: 0.4896 - val_accuracy: 0.8292
Epoch 2/5
94/94 [==============================] - 25s 270ms/step - loss: 0.1855 - accuracy: 0.9433 - val_loss: 0.4111 - val_accuracy: 0.8638
Epoch 3/5
94/94 [==============================] - 24s 252ms/step - loss: 0.1391 - accuracy: 0.9618 - val_loss: 0.4222 - val_accuracy: 0.8549
Epoch 4/5
94/94 [==============================] - 24s 253ms/step - loss: 0.1291 - accuracy: 0.9641 - val_loss: 0.4221 - val_accuracy: 0.8527
Epoch 5/5
94/94 [==============================] - 29s 314ms/step - loss: 0.1018 - accuracy: 0.9775 - val_loss: 0.4564 - val_accuracy: 0.8415


In [19]:
# Save the trained model
InceptionV3model.save('/content/drive/MyDrive/Dataset/InceptionV3_01.h5')

In [20]:
InceptionV3model.evaluate(val_generator)

29/29 [==============================] - 6s 199ms/step - loss: 0.4474 - accuracy: 0.8447


[0.44744163751602173, 0.8447340130805969]